In [ ]:
# config
adapter_output_path = '/content/drive/MyDrive/CLEF2023'
path_to_train_txt = '/content/drive/MyDrive/CLEF2023/pretraining/train.txt'

In [ ]:
# If running in Colab
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install datasets transformers adapter-transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 112.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 115.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import transformers
from transformers import AutoTokenizer

print(transformers.__version__)

4.26.1


## Preparing the dataset

In [ ]:
from datasets import load_dataset

In [ ]:
from datasets import DatasetDict

dataset_split = load_dataset("text", data_files={"train": path_to_train_txt}, split=[
    'train[0%:90%]', 'train[90%:100%]'])

datasets = DatasetDict({'train': dataset_split[0], 'validation': dataset_split[1]})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-8e84dda7c84cfd71/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 261848
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 29094
    })
})

In [ ]:
datasets["train"][10]

{'text': 'Tiempo de tromboplastina parcial activada (aPTT). lupus sensible. exceso de fosfolípidos^Pos neutralización de heparina'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,"administración de vacuna con antígenos polisacáridos capsulares de Streptococcus pneumoniae serotipos daneses 1, 2, 3, 4, 5, 6B, 7F, 8, 9N, 9V, 10A, 11A, 12F, 14, 15B, 17F, 18C, 19A, 19F, 20, 22F, 23F, y 33F como únicos ingredientes"
1,resección de glándula salival
2,laparoscopia de recto
3,"Inserción en tejido subcutáneo y fascia, antebrazo izquierdo, de dispositivo anticonceptivo, abordaje abierto"
4,resección mecánica percutánea automática de disco intervertebral lumbar durante procedimiento de revisión
5,examen de extendido microbiano
6,plicatura perianal de los músculos elevadores del ano y los esfínteres anales
7,"Revisión en vena inferior, de dispositivo de monitorización, abordaje externo"
8,procedimiento en la axila
9,Espermatozoides decabeza redonda/100 espermatozoides


## Masked language modeling

In [ ]:
model_checkpoint = "sultan/BioM-ALBERT-xxlarge"

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"], truncation=True, max_length=512)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/261848 [00:00<?, ? examples/s]

Map:   0%|          | 0/29094 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 261848
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 29094
    })
})

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at sultan/BioM-ALBERT-xxlarge were not used when initializing AlbertForMaskedLM: ['sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import TrainingArguments, AdapterTrainer

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{adapter_output_path}/adapter-{model_name}",
    evaluation_strategy = "epoch",
    learning_rate=1.76e-3,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=256,
    save_steps = 5,
    push_to_hub=False,
    logging_steps=1,
    group_by_length=True,
    # The remove_unised_columns param is important to ensure the dataset labels are properly passed to the model 
    # This was in the example notebook 
    # We don't pass labels explicitly (they are generated by the DataCollator by copying the input_ids), but keep this just to be sure :) 
    remove_unused_columns=False,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers.adapters import PfeifferInvConfig

config = PfeifferInvConfig()

model.add_adapter("biom_spanish", config=config)
model.train_adapter(["biom_spanish"])

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
# to resume training from a checkpoint, specify its path here.
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 261848
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 8192
  Gradient Accumulation steps = 256
  Total optimization steps = 31
  Number of trainable parameters = 2664560
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
tokenizer.decode(lm_datasets['train']['input_ids'][0])

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")